In [1]:
import pandas as pd
import numpy as np

In [2]:
basics_df = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t", low_memory=False)

In [3]:
basics_df =  basics_df.drop('endYear',axis=1)
movies = basics_df.loc[basics_df['titleType']=='movie']
movies = movies.drop('titleType',axis=1)
movies_ok = movies.loc[movies['isAdult']=='0']
movies_ok=movies_ok.drop('isAdult',axis=1)
movies_ok_nat = movies_ok.replace('\\N', np.nan)
movies_clean = movies_ok_nat.dropna()
movies_clean = movies_clean.astype({'runtimeMinutes': int})
movies_clean = movies_clean.astype({'startYear': int})
movies_clean = movies_clean[(movies_clean['runtimeMinutes'] >= 58) & (movies_clean['runtimeMinutes'] <= 270)]
movies_clean = movies_clean[(movies_clean['startYear'] >= 1918) & (movies_clean['startYear'] <= 2021)]

In [4]:
movies_clean.head()

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
2396,tt0002423,Passion,Madame DuBarry,1919,85,"Biography,Drama,Romance"
5013,tt0005076,Charley's Aunt,Charley's Aunt,1925,80,Comedy
6979,tt0007068,Mind the Paint Girl,Mind the Paint Girl,1919,60,"Drama,Romance"
7328,tt0007422,Das Tagebuch des Dr. Hart,Das Tagebuch des Dr. Hart,1918,82,"Drama,Romance"
7449,tt0007547,Der Weg des Todes,Der Weg des Todes,1918,75,Drama


In [5]:
ratings_df = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t")

In [6]:
movies = pd.merge(movies_clean,ratings_df,how='inner')
movies_7 = movies.loc[movies['averageRating']>=7.0]
movies_7_rating = movies_7[movies_7['numVotes'] >= 1000]

In [7]:
movies_7_rating.head()

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
11,tt0008879,The Outlaw and His Wife,Berg-Ejvind och hans hustru,1918,136,Drama,7.2,1784
82,tt0009893,The Oyster Princess,Die Austernprinzessin,1919,60,Comedy,7.2,1817
88,tt0009937,Blind Husbands,Blind Husbands,1919,99,"Drama,Romance",7.0,1470
91,tt0009968,Broken Blossoms,Broken Blossoms or The Yellow Man and the Girl,1919,90,"Drama,Romance",7.3,10029
124,tt0010247,Sir Arne's Treasure,Herr Arnes pengar,1919,122,"Drama,History",7.2,1357


In [8]:
movie_for_ml = movies_7_rating.drop(columns=['primaryTitle','runtimeMinutes','averageRating','numVotes'],axis =1)

In [9]:
movie_for_ml.reset_index(drop=True, inplace=True)
movie_for_ml

,tconst,originalTitle,startYear,genres
0,tt0008879,Berg-Ejvind och hans hustru,1918,Drama
1,tt0009893,Die Austernprinzessin,1919,Comedy
2,tt0009937,Blind Husbands,1919,"Drama,Romance"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,1919,"Drama,Romance"
4,tt0010247,Herr Arnes pengar,1919,"Drama,History"
...,...,...,...,...
11649,tt9882084,Chasing Happiness,2019,"Biography,Documentary,Music"
11650,tt9886872,Munthiri Monchan,2019,"Comedy,Romance"
11651,tt9900782,Kaithi,2019,"Action,Crime,Thriller"
11652,tt9902160,Herself,2020,Drama


In [10]:
movie_for_ml = movie_for_ml.astype({'startYear': str})
movie_for_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11654 entries, 0 to 11653
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tconst         11654 non-null  object
 1   originalTitle  11654 non-null  object
 2   startYear      11654 non-null  object
 3   genres         11654 non-null  object
dtypes: object(4)
memory usage: 364.3+ KB


In [11]:
movie_for_ml['data'] = movie_for_ml['startYear'] + ',' + movie_for_ml['genres']
movie_for_ml

,tconst,originalTitle,startYear,genres,data
0,tt0008879,Berg-Ejvind och hans hustru,1918,Drama,"1918,Drama"
1,tt0009893,Die Austernprinzessin,1919,Comedy,"1919,Comedy"
2,tt0009937,Blind Husbands,1919,"Drama,Romance","1919,Drama,Romance"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,1919,"Drama,Romance","1919,Drama,Romance"
4,tt0010247,Herr Arnes pengar,1919,"Drama,History","1919,Drama,History"
...,...,...,...,...,...
11649,tt9882084,Chasing Happiness,2019,"Biography,Documentary,Music","2019,Biography,Documentary,Music"
11650,tt9886872,Munthiri Monchan,2019,"Comedy,Romance","2019,Comedy,Romance"
11651,tt9900782,Kaithi,2019,"Action,Crime,Thriller","2019,Action,Crime,Thriller"
11652,tt9902160,Herself,2020,Drama,"2020,Drama"


In [12]:
movie_for_ml.drop(columns=['startYear','genres'],axis=1,inplace=True)
movie_for_ml

,tconst,originalTitle,data
0,tt0008879,Berg-Ejvind och hans hustru,"1918,Drama"
1,tt0009893,Die Austernprinzessin,"1919,Comedy"
2,tt0009937,Blind Husbands,"1919,Drama,Romance"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,Drama,Romance"
4,tt0010247,Herr Arnes pengar,"1919,Drama,History"
...,...,...,...
11649,tt9882084,Chasing Happiness,"2019,Biography,Documentary,Music"
11650,tt9886872,Munthiri Monchan,"2019,Comedy,Romance"
11651,tt9900782,Kaithi,"2019,Action,Crime,Thriller"
11652,tt9902160,Herself,"2020,Drama"


In [13]:
temp_clusters = pd.read_csv('../data/movies_for_ml.csv.zip')
temp_clusters

,tconst,originalTitle,startYear,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,cluster
0,tt0008879,Berg-Ejvind och hans hustru,-2.974130,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,5
1,tt0009893,Die Austernprinzessin,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,1.587382,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,24
2,tt0009937,Blind Husbands,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,4
4,tt0010247,Herr Arnes pengar,-2.934086,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11649,tt9882084,Chasing Happiness,1.070327,-0.391523,-0.325588,-0.215457,3.179564,-0.629968,-0.436904,2.801589,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,27
11650,tt9886872,Munthiri Monchan,1.070327,-0.391523,-0.325588,-0.215457,-0.314509,1.587382,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,2.157438,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,29
11651,tt9900782,Kaithi,1.070327,2.554127,-0.325588,-0.215457,-0.314509,-0.629968,2.288833,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,3.138634,-0.214802,-0.11648,2
11652,tt9902160,Herself,1.110371,-0.391523,-0.325588,-0.215457,-0.314509,-0.629968,-0.436904,-0.356940,...,-0.149567,-0.24403,-0.045427,-0.463513,-0.150166,-0.160873,-0.318610,-0.214802,-0.11648,5


In [14]:
movie_for_ml['cluster'] = temp_clusters['cluster']
movie_for_ml['data'] = movie_for_ml['data'].apply(str.lower)
movie_for_ml


,tconst,originalTitle,data,cluster
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5
1,tt0009893,Die Austernprinzessin,"1919,comedy",24
2,tt0009937,Blind Husbands,"1919,drama,romance",4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18
...,...,...,...,...
11649,tt9882084,Chasing Happiness,"2019,biography,documentary,music",27
11650,tt9886872,Munthiri Monchan,"2019,comedy,romance",29
11651,tt9900782,Kaithi,"2019,action,crime,thriller",2
11652,tt9902160,Herself,"2020,drama",5


In [15]:
movie_for_ml.to_csv('../data/movies_merged.csv.zip',index=False, compression='zip')

In [16]:
movies_merged = pd.read_csv('../data/movies_merged.csv.zip')
movies_merged

,tconst,originalTitle,data,cluster
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5
1,tt0009893,Die Austernprinzessin,"1919,comedy",24
2,tt0009937,Blind Husbands,"1919,drama,romance",4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18
...,...,...,...,...
11649,tt9882084,Chasing Happiness,"2019,biography,documentary,music",27
11650,tt9886872,Munthiri Monchan,"2019,comedy,romance",29
11651,tt9900782,Kaithi,"2019,action,crime,thriller",2
11652,tt9902160,Herself,"2020,drama",5


In [17]:
crew_df = pd.read_csv("https://datasets.imdbws.com/title.crew.tsv.gz", sep="\t")

In [18]:
movies_crew = pd.merge(movies_merged,crew_df,how='inner')

In [19]:
movies_crew

,tconst,originalTitle,data,cluster,directors,writers
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5,nm0803705,"nm0803705,nm0039377,nm2642082"
1,tt0009893,Die Austernprinzessin,"1919,comedy",24,nm0523932,"nm0473134,nm0523932"
2,tt0009937,Blind Husbands,"1919,drama,romance",4,nm0002233,"nm0002233,nm0239769"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4,nm0000428,"nm0121885,nm0000428"
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18,nm0830249,"nm0481248,nm0596410,nm0830249"
...,...,...,...,...,...,...
11649,tt9882084,Chasing Happiness,"2019,biography,documentary,music",27,nm3039295,\N
11650,tt9886872,Munthiri Monchan,"2019,comedy,romance",29,nm9295244,"nm10525626,nm10525627"
11651,tt9900782,Kaithi,"2019,action,crime,thriller",2,nm7992231,"nm7992231,nm7807469"
11652,tt9902160,Herself,"2020,drama",5,nm1630273,"nm7434342,nm0132696"


In [20]:
movies_crew = movies_crew.replace('\\N', 'None')
movies_crew

,tconst,originalTitle,data,cluster,directors,writers
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5,nm0803705,"nm0803705,nm0039377,nm2642082"
1,tt0009893,Die Austernprinzessin,"1919,comedy",24,nm0523932,"nm0473134,nm0523932"
2,tt0009937,Blind Husbands,"1919,drama,romance",4,nm0002233,"nm0002233,nm0239769"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4,nm0000428,"nm0121885,nm0000428"
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18,nm0830249,"nm0481248,nm0596410,nm0830249"
...,...,...,...,...,...,...
11649,tt9882084,Chasing Happiness,"2019,biography,documentary,music",27,nm3039295,None
11650,tt9886872,Munthiri Monchan,"2019,comedy,romance",29,nm9295244,"nm10525626,nm10525627"
11651,tt9900782,Kaithi,"2019,action,crime,thriller",2,nm7992231,"nm7992231,nm7807469"
11652,tt9902160,Herself,"2020,drama",5,nm1630273,"nm7434342,nm0132696"


In [21]:
principals_df = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t")

In [22]:
principals_df.head(10)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
6,tt0000003,2,nm1770680,producer,producer,\N
7,tt0000003,3,nm1335271,composer,\N,\N
8,tt0000003,4,nm5442200,editor,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N


In [23]:
actors = principals_df.loc[(principals_df['category']=='actor')|(principals_df['category']=='actress')]

In [24]:
actors.drop(columns=['ordering', 'category', 'job', 'characters'],axis=1,inplace=True)
actors

/Users/guillaumearp/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,tconst,nconst
11,tt0000005,nm0443482
12,tt0000005,nm0653042
16,tt0000007,nm0179163
17,tt0000007,nm0183947
21,tt0000008,nm0653028
...,...,...
47575099,tt9916856,nm10538647
47575104,tt9916880,nm1483166
47575105,tt9916880,nm0254176
47575106,tt9916880,nm0286175


In [25]:
actors = (actors.groupby(['tconst'])['nconst'].apply(list)).reset_index()
actors

,tconst,nconst
0,tt0000005,"[nm0443482, nm0653042]"
1,tt0000007,"[nm0179163, nm0183947]"
2,tt0000008,[nm0653028]
3,tt0000009,"[nm0063086, nm0183823, nm1309758]"
4,tt0000011,[nm3692297]
...,...,...
4704146,tt9916848,"[nm5519557, nm8825009, nm5262613, nm8690065]"
4704147,tt9916850,"[nm5519557, nm8825009, nm5262613, nm8690065]"
4704148,tt9916852,"[nm5519557, nm8825009, nm5262613, nm8690065]"
4704149,tt9916856,"[nm3394271, nm10538650, nm10538646, nm10538647]"


In [26]:
movies_all = pd.merge(movies_crew,actors,how='inner')
movies_all

,tconst,originalTitle,data,cluster,directors,writers,nconst
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5,nm0803705,"nm0803705,nm0039377,nm2642082","[nm0803705, nm0258611, nm0252476, nm0014363]"
1,tt0009893,Die Austernprinzessin,"1919,comedy",24,nm0523932,"nm0473134,nm0523932","[nm0418131, nm0652661, nm0509573, nm0266254]"
2,tt0009937,Blind Husbands,"1919,drama,romance",4,nm0002233,"nm0002233,nm0239769","[nm0208659, nm0082537, nm0002233, nm0332998]"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4,nm0000428,"nm0121885,nm0000428","[nm0782999, nm0001273, nm0001932, nm0187981, n..."
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18,nm0830249,"nm0481248,nm0596410,nm0830249","[nm0830862, nm0074129, nm0526234, nm0632163]"
...,...,...,...,...,...,...,...
10602,tt9875128,Halahal,"2020,crime,drama,mystery",22,nm6095820,"nm11925461,nm6960435,nm5113220","[nm6633565, nm2286188, nm4949471, nm0451561]"
10603,tt9886872,Munthiri Monchan,"2019,comedy,romance",29,nm9295244,"nm10525626,nm10525627","[nm4270963, nm9686121, nm1368581, nm0222181]"
10604,tt9900782,Kaithi,"2019,action,crime,thriller",2,nm7992231,"nm7992231,nm7807469","[nm1912683, nm1230844, nm6030469, nm6998719]"
10605,tt9902160,Herself,"2020,drama",5,nm1630273,"nm7434342,nm0132696","[nm9450235, nm7434342, nm9007992, nm2988542]"


In [27]:
movies_all.rename(columns={'nconst': 'actors'}, inplace=True)
movies_all

,tconst,originalTitle,data,cluster,directors,writers,actors
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5,nm0803705,"nm0803705,nm0039377,nm2642082","[nm0803705, nm0258611, nm0252476, nm0014363]"
1,tt0009893,Die Austernprinzessin,"1919,comedy",24,nm0523932,"nm0473134,nm0523932","[nm0418131, nm0652661, nm0509573, nm0266254]"
2,tt0009937,Blind Husbands,"1919,drama,romance",4,nm0002233,"nm0002233,nm0239769","[nm0208659, nm0082537, nm0002233, nm0332998]"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4,nm0000428,"nm0121885,nm0000428","[nm0782999, nm0001273, nm0001932, nm0187981, n..."
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18,nm0830249,"nm0481248,nm0596410,nm0830249","[nm0830862, nm0074129, nm0526234, nm0632163]"
...,...,...,...,...,...,...,...
10602,tt9875128,Halahal,"2020,crime,drama,mystery",22,nm6095820,"nm11925461,nm6960435,nm5113220","[nm6633565, nm2286188, nm4949471, nm0451561]"
10603,tt9886872,Munthiri Monchan,"2019,comedy,romance",29,nm9295244,"nm10525626,nm10525627","[nm4270963, nm9686121, nm1368581, nm0222181]"
10604,tt9900782,Kaithi,"2019,action,crime,thriller",2,nm7992231,"nm7992231,nm7807469","[nm1912683, nm1230844, nm6030469, nm6998719]"
10605,tt9902160,Herself,"2020,drama",5,nm1630273,"nm7434342,nm0132696","[nm9450235, nm7434342, nm9007992, nm2988542]"


In [28]:
movies_all['actors'] = movies_all['actors'].str.join(',')

In [29]:
movies_all

,tconst,originalTitle,data,cluster,directors,writers,actors
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama",5,nm0803705,"nm0803705,nm0039377,nm2642082","nm0803705,nm0258611,nm0252476,nm0014363"
1,tt0009893,Die Austernprinzessin,"1919,comedy",24,nm0523932,"nm0473134,nm0523932","nm0418131,nm0652661,nm0509573,nm0266254"
2,tt0009937,Blind Husbands,"1919,drama,romance",4,nm0002233,"nm0002233,nm0239769","nm0208659,nm0082537,nm0002233,nm0332998"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance",4,nm0000428,"nm0121885,nm0000428","nm0782999,nm0001273,nm0001932,nm0187981,nm0397..."
4,tt0010247,Herr Arnes pengar,"1919,drama,history",18,nm0830249,"nm0481248,nm0596410,nm0830249","nm0830862,nm0074129,nm0526234,nm0632163"
...,...,...,...,...,...,...,...
10602,tt9875128,Halahal,"2020,crime,drama,mystery",22,nm6095820,"nm11925461,nm6960435,nm5113220","nm6633565,nm2286188,nm4949471,nm0451561"
10603,tt9886872,Munthiri Monchan,"2019,comedy,romance",29,nm9295244,"nm10525626,nm10525627","nm4270963,nm9686121,nm1368581,nm0222181"
10604,tt9900782,Kaithi,"2019,action,crime,thriller",2,nm7992231,"nm7992231,nm7807469","nm1912683,nm1230844,nm6030469,nm6998719"
10605,tt9902160,Herself,"2020,drama",5,nm1630273,"nm7434342,nm0132696","nm9450235,nm7434342,nm9007992,nm2988542"


In [30]:
movies_all['data'] = movies_all['data'] + ',' + movies_all['directors'] + ',' + movies_all['writers'] + ',' + movies_all['actors']
movies_all

,tconst,originalTitle,data,cluster,directors,writers,actors
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama,nm0803705,nm0803705,nm0039377,nm264...",5,nm0803705,"nm0803705,nm0039377,nm2642082","nm0803705,nm0258611,nm0252476,nm0014363"
1,tt0009893,Die Austernprinzessin,"1919,comedy,nm0523932,nm0473134,nm0523932,nm04...",24,nm0523932,"nm0473134,nm0523932","nm0418131,nm0652661,nm0509573,nm0266254"
2,tt0009937,Blind Husbands,"1919,drama,romance,nm0002233,nm0002233,nm02397...",4,nm0002233,"nm0002233,nm0239769","nm0208659,nm0082537,nm0002233,nm0332998"
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance,nm0000428,nm0121885,nm00004...",4,nm0000428,"nm0121885,nm0000428","nm0782999,nm0001273,nm0001932,nm0187981,nm0397..."
4,tt0010247,Herr Arnes pengar,"1919,drama,history,nm0830249,nm0481248,nm05964...",18,nm0830249,"nm0481248,nm0596410,nm0830249","nm0830862,nm0074129,nm0526234,nm0632163"
...,...,...,...,...,...,...,...
10602,tt9875128,Halahal,"2020,crime,drama,mystery,nm6095820,nm11925461,...",22,nm6095820,"nm11925461,nm6960435,nm5113220","nm6633565,nm2286188,nm4949471,nm0451561"
10603,tt9886872,Munthiri Monchan,"2019,comedy,romance,nm9295244,nm10525626,nm105...",29,nm9295244,"nm10525626,nm10525627","nm4270963,nm9686121,nm1368581,nm0222181"
10604,tt9900782,Kaithi,"2019,action,crime,thriller,nm7992231,nm7992231...",2,nm7992231,"nm7992231,nm7807469","nm1912683,nm1230844,nm6030469,nm6998719"
10605,tt9902160,Herself,"2020,drama,nm1630273,nm7434342,nm0132696,nm945...",5,nm1630273,"nm7434342,nm0132696","nm9450235,nm7434342,nm9007992,nm2988542"


In [34]:
movies_final = movies_all.drop(columns=['directors','writers','actors'],axis=1)
movies_final

,tconst,originalTitle,data,cluster
0,tt0008879,Berg-Ejvind och hans hustru,"1918,drama,nm0803705,nm0803705,nm0039377,nm264...",5
1,tt0009893,Die Austernprinzessin,"1919,comedy,nm0523932,nm0473134,nm0523932,nm04...",24
2,tt0009937,Blind Husbands,"1919,drama,romance,nm0002233,nm0002233,nm02397...",4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,"1919,drama,romance,nm0000428,nm0121885,nm00004...",4
4,tt0010247,Herr Arnes pengar,"1919,drama,history,nm0830249,nm0481248,nm05964...",18
...,...,...,...,...
10602,tt9875128,Halahal,"2020,crime,drama,mystery,nm6095820,nm11925461,...",22
10603,tt9886872,Munthiri Monchan,"2019,comedy,romance,nm9295244,nm10525626,nm105...",29
10604,tt9900782,Kaithi,"2019,action,crime,thriller,nm7992231,nm7992231...",2
10605,tt9902160,Herself,"2020,drama,nm1630273,nm7434342,nm0132696,nm945...",5


In [36]:
def clear(city):
    city_split = city.split(',')
    
    merged_city = " ".join(city_split)
    return merged_city

In [38]:
movies_final['data'] = movies_final['data'].apply(clear)
movies_final

,tconst,originalTitle,data,cluster
0,tt0008879,Berg-Ejvind och hans hustru,1918 drama nm0803705 nm0803705 nm0039377 nm264...,5
1,tt0009893,Die Austernprinzessin,1919 comedy nm0523932 nm0473134 nm0523932 nm04...,24
2,tt0009937,Blind Husbands,1919 drama romance nm0002233 nm0002233 nm02397...,4
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl,1919 drama romance nm0000428 nm0121885 nm00004...,4
4,tt0010247,Herr Arnes pengar,1919 drama history nm0830249 nm0481248 nm05964...,18
...,...,...,...,...
10602,tt9875128,Halahal,2020 crime drama mystery nm6095820 nm11925461 ...,22
10603,tt9886872,Munthiri Monchan,2019 comedy romance nm9295244 nm10525626 nm105...,29
10604,tt9900782,Kaithi,2019 action crime thriller nm7992231 nm7992231...,2
10605,tt9902160,Herself,2020 drama nm1630273 nm7434342 nm0132696 nm945...,5


In [35]:
movies_final.to_csv('../data/movies_merged.csv.zip',index=False, compression='zip')